In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk
from os.path import splitext

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD

import custom_loss_functions as cust_loss
from load_data import get_ID_list, DataGenerator_raw

Using TensorFlow backend.


## Model

In [2]:
def simple_model():
    """Define simple model"""
    # create model
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(96,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # show info
    model.summary()
    # compile model
    model.compile(loss = cust_loss.mse_wrap_angle,
                  optimizer = 'adam',
                  metrics = [cust_loss.mae_wrap_angle])
    return model

## Training

In [3]:
# define training parameters
batch_size = 32
num_epochs = 10

In [4]:
# define filename of file containing dataset
filename = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/database_raw.h5'

In [5]:
#train_test_split

# list of filename substrings of files to ignore, e.g. 'NFCHOA', 'pos10'
ignore_list = []# ['LWFS','R006','M006','M027','R027','M013','R013']
# dictionary containing train, validation and test subset IDs
partition = {'train':[], 'validation':[], 'test':[]}
# load data indices
#list_ID, n_rows = get_ID_list(filelist, ignore_list, n_rows=720000)
list_ID = np.arange(160*360*100*20,dtype='int32')
# split data set in training, validation and test data
partition['train'], partition['test'] = train_test_split(list_ID, shuffle=True, test_size=0.2)
partition['train'], partition['validation'] = train_test_split(partition['train'], shuffle=True, test_size=0.2)
del list_ID

In [8]:
feature_label = ['test']
target_label = ['test2']

In [ ]:
# model.fit_generator on batches of dataset

# define generators
params = {'dim': len(feature_label),
          'batch_size': batch_size,
          'filename': filename,
          'shuffle': True,
          'feature_label': feature_label,
          'target_label': target_label,
          'keys': ['feature_data', 'target_data']
         }
train_batch_generator = DataGenerator_raw(partition['train'], **params)
valid_batch_generator = DataGenerator_raw(partition['validation'], **params)

model = simple_model()

history = model.fit_generator(generator = train_batch_generator,
                              #steps_per_epoch = (num_train_samples) // batch_size),
                              epochs = num_epochs,
                              verbose = 1,
                              validation_data = valid_batch_generator,
                              #validation_steps = (num_valid_samples) // batch_size),
                              use_multiprocessing = True,
                              workers = 4)

W0920 01:51:54.604614 140313577035584 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0920 01:51:54.641300 140313577035584 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0920 01:51:54.652419 140313577035584 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0920 01:51:54.711464 140313577035584 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimize

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               12416     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 29,057
Trainable params: 29,057
Non-trainable params: 0
_________________________________________________________________


W0920 01:51:55.070500 140313577035584 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0920 01:51:55.194378 140313577035584 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/10


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/feliximmohr/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/feliximmohr/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/utils/data_utils.py", line 565, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/utils/data_utils.py", line 548, in <lambda>
    initargs=(seqs,))
  File "/home/feliximmohr/anaconda3/lib/python3.7/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/home/feliximmohr/anaconda3/lib/python3.7/multiprocessing/pool.py", line 176, in __init__
    self._repopulate_pool()
  File "/home/feliximmohr/anaconda3/lib/python3.7/multiprocessing/pool.py", line 241, in _repopulate_pool
    w.start()
  File "/

In [ ]:
# normalization !!! TODO

# define scaler
minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
minmax_scaler.partial_fit(data[feature_label])
#minmax_scaler.partial_fit(test_data[feature_label])

std_scale = preprocessing.StandardScaler().fit(data[feature_label])

# perform scaling
train_minmax = minmax_scaler.transform(train[feature_label])
test_minmax = minmax_scaler.transform(test[feature_label])
train_std = std_scale.transform(train[feature_label])
test_std = std_scale.transform(test[feature_label])

## Evaluation

In [ ]:
# evaluate model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test mae w/o wrap:', score[1])
print('Test mae w wrap:', score[2])

# plot train history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Save

In [ ]:
# Save history to json file
import json
with open('file.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

In [ ]:
partition['train']